In [ ]:
import gensim
from bs4 import BeautifulSoup
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk import download
download('wordnet')
download('averaged_perceptron_tagger')
from nltk import WordNetLemmatizer, pos_tag
from nltk.corpus import wordnet
from gensim.models import Phrases
from gensim.models.coherencemodel import CoherenceModel
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
import re

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
Mounted at /content/drive


In [ ]:
stops = pd.read_csv('/content/drive/My Drive/datasets/articles/monthPre/final/completo/preprocessedCompletoComBigrams.csv')

In [ ]:
mes1 = stops[stops.month == 1]

In [ ]:
news = stops[stops.source == 'huffpost']

In [ ]:
idx = 0 
for doc in mes1.itertuples():
  for word in doc.content.split(' '):
    if word == 'george_floyd':
      print(doc.link)
      idx +=1
      break


In [ ]:
links = stops.link

In [ ]:
news = stops.content

In [ ]:
links[500085]

In [ ]:
news[500085]

In [ ]:
stops = pd.read_csv('/content/drive/My Drive/datasets/articles/share/stops/stops.txt')
stopsList = stops.document
stops = [word for word in stopsList]
for w in STOPWORDS:
  stops.append(w)

In [ ]:
class PreProcessing(object):

    def __init__(self):
        self.__wordNet = WordNetLemmatizer()
       
    
    def __clearHTML(self, doc):
        soup = BeautifulSoup(doc, 'lxml')
        for tag in soup.find_all('code'):
            tag.decompose()
        return self.__removeCode(soup.get_text(), repeat = 4)
    
    def __getPOS(self, token):
        tag = pos_tag([token])[0][1][0].upper()
        tagDict = {
            "J": wordnet.ADJ,
            "N": wordnet.NOUN,
            "V": wordnet.VERB,
            "R": wordnet.ADV
        }
        return tagDict.get(tag, wordnet.NOUN)


    def __removeCode(self, content, repeat = 1):
        for i in range(0,repeat):
            #AS VEZES, quando executa uma vez sobra algum lixo ainda, por isso o repeat
            content = re.sub("\w*\.\w*\.\w*", "", content) # retira palavras seguidas de pontos nesse formato ex: place.googletag.cmd
            content = re.sub(r"\{[^{}]*\}", "", content) #retira palavras entre { }
            content = re.sub(r"\w*\.\w+\([^()]*\)", "", content) ##retira palavras com ponto que seguem de ( ) ex: document.function(...)
            content = re.sub(r"\w+\([^()]*\)", "", content) ##retira palavras que seguem de ( ) ex: function(...)
        return content

    def __applyNLP(self, doc):
        newDoc = []
        for token in simple_preprocess(doc, deacc=True):
          if self.__getPOS(token) == 'v': ## verbo tem retorno do POS como 'v', caso queira remover outro POS é só acrescentar a sigla na lista
            continue
          #print(self.__getPOS(token))
          token = self.__wordNet.lemmatize(token, pos=self.__getPOS(token))
          if token not in stops and len(token) > 2:
              newDoc.append(token)
        return newDoc
    
    def __cleaning(self, corpus):
        cleanCorpus = []
        for doc in corpus:
            doc = self.__clearHTML(doc)
            doc = self.__applyNLP(doc)
            cleanCorpus.append(doc)
        return cleanCorpus
    
    def __makeBigrams(self, corpus):
        # Train Phrases model
        bigramModel.add_vocab(corpus)
        # Make bi-grams
        
        preProcessedCorpus = []
        for doc in corpus:
          #doc = bigramModel[doc]
          if self.__join:
              doc = ' '.join(doc)
          preProcessedCorpus.append(doc)
        
        return  corpus, bigramModel

    def execute(self, corpus, bModel, join=False):
        self.__join = join
        self.__bigramModel = bModel
        return self.__makeBigrams(self.__cleaning(corpus))


In [ ]:
def remove(content):
  newDocs = []
  for doc in content:
    docLine = ''
    aux = []
    for word in doc.split(','):
      word = "".join(c  for c in word if c not in ["'", '"', '[', ']', " "] )
      #docLine += word+' '
      aux.append(word)
    newDocs.append(aux)
  return newDocs

    



In [ ]:
nc = remove(test.content)

In [ ]:
bigramModel = Phrases()

In [ ]:
preProcessing = PreProcessing()

In [ ]:
bigramModel = Phrases()
bigramModel = bigramModel.load('/content/drive/My Drive/datasets/articles/bmodel/bmodel.bin')

In [ ]:
test = pd.read_csv('/content/drive/My Drive/datasets/articles/monthPre/monthpre2.csv')

In [ ]:
#dataset divided by month
for i in range(1,13):
  print('Mes:', i)
  dfatual  = pd.read_csv('/content/drive/My Drive/datasets/articles/datasetsDivided/month{}.csv'.format(i))
  dfatual = dfatual[dfatual['content'].notna()]
  
  newCorpus, bigramModel = preProcessing.execute(dfatual.content, bigramModel, join = True )
  #print(newCorpus[0])
  print('Salvando modelo de bigrams...')
  bigramModel.save(fname_or_handle= '/content/drive/My Drive/datasets/articles/bmodel/bmodel.bin', )

  dfatual['content'] = newCorpus
  print('Salvando csv')
  dfatual.to_csv('/content/drive/My Drive/datasets/articles/monthPre/monthpre{}.csv'.format(i))

de sexta dia 19
Mes: 1
Salvando modelo de bigrams...
Salvando csv
Mes: 2
Salvando modelo de bigrams...
Salvando csv
Mes: 3
Salvando modelo de bigrams...
Salvando csv
Mes: 4